In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from data_utils import get_data
import os
import numpy as np

In [11]:
# Extraire les images et les labels
images, labels = [], []

events = ['cards', 
          'center'
          #'free-kick'
         ]

train_images, train_labels = get_data(folder='train', events=events)

In [ ]:
print(train_images[0])
print(train_labels[0])